In [1]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import mediapipe as mp
import cv2


Format Data

In [4]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
def extract_coordinates():
    rows = []
    #creating empty file in folder, I added the start_time in the name of the csv file, so that if a symbol appears many times in a video, it will still be created in two different csv files, just that they will have different starting times
    csv_file = f"demo.csv"
    # if os.path.exists(csv_file):
    #     return 



# Setup CSV File for the videos
# 21 right hand landmarks, 21 left hand landmarks, 33 pose landmarks
    num_coords = 21 + 21 + 33
    landmarks = []
    for val in range(1, num_coords+1):
        landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
    print("Initialized an empty landmarks of size:", len(landmarks))

    with open(csv_file, mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)
    
    

#working with each video
    cap = cv2.VideoCapture(0)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
    # Read until video is completed
    else: 
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while(cap.isOpened()):  
                # Capture frame-by-frame
                ret, image = cap.read() 
                if ret == True:
                    # the BGR image to RGB.
                    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
                    # To improve performance, optionally mark the image as not writeable to
                    # pass by reference.
                    image.flags.writeable = False

                    #!results contains all the information about the image, in this case, we are looking at hands
                    results = holistic.process(image)
                    
                    
                    # Draw the hand annotations on the image.
                    image.flags.writeable = True
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    
                    # Display the resulting frame
                    # Right hand
                    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                            )

                    # Left Hand
                    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                            )

                    # Pose Detections
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                            )
                    cv2.imshow('Frame',image)
                    # Press Q on keyboard to  exitx
                    if cv2.waitKey(25) & 0xFF == ord('q'):
                        frame_width = int(cap.get(3)) 
                        frame_height = int(cap.get(4)) 
                        
                        size = (frame_width, frame_height) 
                        result = cv2.VideoWriter('test.mp4',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, size) 
                        break
                    # Export coordinates
                    try:
                        # Extract Pose landmarks
                        if results.pose_landmarks:
                            pose = results.pose_landmarks.landmark
                            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
                        else:
                            # continue
                            pose_row=list(np.array([[0,0,0,0] for i in range(33)]).flatten())
                        # Extract hands landmarks
                        if results.right_hand_landmarks:
                            right_hand = results.right_hand_landmarks.landmark
                            right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
                        else:
                            #If no right hand detected, then it writes 0 to the CSV file
                            right_hand_row = list(np.array([[0,0,0,0] for i in range(21)]).flatten())
                        if results.left_hand_landmarks:
                            left_hand = results.left_hand_landmarks.landmark
                            left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
                        else:
                            #If no left hand detected, then it writes 0 to the CSV file
                            left_hand_row = list(np.array([[0,0,0,0] for i in range(21)]).flatten())

                        # Concate rows
                        row = pose_row + right_hand_row + left_hand_row
                        rows.append(row)

                        # Export to CSV
                        

                    except Exception as e:
                        print(e)
                        break
                         
                else:
                    break

            # When everything done, release the video capture object
            cap.release() 


            # Closes all the frames
            cv2.destroyAllWindows()
            with open(csv_file, mode='a', newline='') as f:
                            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                            for row in rows:
                                csv_writer.writerow(row) 
            # return results

In [5]:
extract_coordinates()

Initialized an empty landmarks of size: 300


OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [4]:
class ListNode:
    def __init__(self, value=0, next=None):
        self.value = value
        self.next = next

def reverse_linked_list(lst):
    new_lst = None
    while lst != None:
        temp_list = ListNode(lst.value, new_lst)
        new_lst = temp_list
        lst = lst.next
    return new_lst